In [236]:
import os
import cv2
import random
import shutil
import scipy.io
import numpy as np 
import pandas as pd
from pathlib import Path
from collections import Counter
from scipy.linalg import inv, svd
from sklearn.neighbors import NearestNeighbors
from skfeature.function.similarity_based import fisher_score
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [238]:
# Set dataset path (Google Drive or local folder)
d_p = Path("C:\\Users\\prade\\OneDrive\\Desktop\\PROJECT\\Datasets\\EXT_YALE_B")  # Change this based on your actual folder path
Total_image_per_dataset=60
Num_of_train_image=15
Num_of_test_image=Total_image_per_dataset-Num_of_train_image
Total_class=38

In [204]:
# === Create Directories if They Do Not Exist ===
P = []  # Training set
t = []  # Test set
shutil.rmtree("C:\\Users\\prade\\OneDrive\\Desktop\\PROJECT\\Datasets\\V3-Y\\P", ignore_errors=True)
shutil.rmtree("C:\\Users\\prade\\OneDrive\\Desktop\\PROJECT\\Datasets\\V3-Y\\t", ignore_errors=True)
train_labels = []
test_labels = []
train_path="C:\\Users\\prade\\OneDrive\\Desktop\\PROJECT\\Datasets\\V3-Y\\P"
test_path="C:\\Users\\prade\\OneDrive\\Desktop\\PROJECT\\Datasets\\V3-Y\\t"
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)
IMAGE_SIZE = (32, 32)
,
# ==== Loop through each subfolder in order ====
subfolders = sorted(os.listdir(d_p))
processed_subfolders = 0
for folder in subfolders:
    person_path = os.path.join(d_p, folder)

    if os.path.isdir(person_path):
        # Filter only valid image files (to avoid hidden/system files)
        images = [f for f in os.listdir(person_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        if len(images) >= Num_of_test_image:  
            test_images = random.sample(images, Num_of_test_image)
            train_images = [img for img in images if img not in test_images]
        else:
            # If fewer than 3 images, use all for training
            test_images = []
            train_images = images

        # ==== Processing Training Set ====
        if train_images:
            os.makedirs(os.path.join(train_path, folder), exist_ok=True)
            for img in train_images:
                img_path = os.path.join(person_path, img)
                #print("train", img_path)
                img_data = cv2.imread(img_path)
                if img_data is not None:
                    img_data = cv2.resize(img_data, IMAGE_SIZE)

                    # Flatten and normalize
                    img_vector = img_data.flatten().astype(np.float32)
                    norm = np.linalg.norm(img_vector, ord=2)
                    if norm != 0:
                        img_vector = img_vector / norm

                    # Append to training set
                    P.append(img_vector)
                    train_labels.append(folder)
                    # Copy image to train directory
                    shutil.copy(img_path, os.path.join(train_path, folder, img))

        # ==== Processing Test Set ====
        if test_images:
            os.makedirs(os.path.join(test_path, folder), exist_ok=True)
            for img in test_images:
                img_path = os.path.join(person_path, img)
                #print("test", img_path)
                img_data = cv2.imread(img_path)
                if img_data is not None:
                    img_data = cv2.resize(img_data, IMAGE_SIZE)

                    # Flatten and normalize
                    img_vector = img_data.flatten().astype(np.float32)
                    norm = np.linalg.norm(img_vector, ord=2)
                    if norm != 0:
                        img_vector = img_vector / norm

                    # Append to test set
                    t.append(img_vector)
                    test_labels.append(folder)
                    os.makedirs(os.path.join(person_path, folder), exist_ok=True)
                    # Copy image to test directory
                    shutil.copy(img_path, os.path.join(test_path, folder, img))

        processed_subfolders += 1

# Convert lists to numpy arrays
P = np.array(P)
t = np.array(t)

# Print shapes
print(f"Training set shape (P): {P.shape}")
print(f"Test set shape (t): {t.shape}")

# Convert labels to numpy arrays if needed
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

print(train_labels)
print(test_labels)
print(f"Training labels shape: {train_labels.shape}")
print(f"Test labels shape: {test_labels.shape}")

# Check number of processed subfolders
print(f"Number of processed subfolders: {processed_subfolders}")

Training set shape (P): (570, 3072)
Test set shape (t): (1710, 3072)
['01' '01' '01' '01' '01' '01' '01' '01' '01' '01' '01' '01' '01' '01'
 '01' '02' '02' '02' '02' '02' '02' '02' '02' '02' '02' '02' '02' '02'
 '02' '02' '03' '03' '03' '03' '03' '03' '03' '03' '03' '03' '03' '03'
 '03' '03' '03' '04' '04' '04' '04' '04' '04' '04' '04' '04' '04' '04'
 '04' '04' '04' '04' '05' '05' '05' '05' '05' '05' '05' '05' '05' '05'
 '05' '05' '05' '05' '05' '06' '06' '06' '06' '06' '06' '06' '06' '06'
 '06' '06' '06' '06' '06' '06' '07' '07' '07' '07' '07' '07' '07' '07'
 '07' '07' '07' '07' '07' '07' '07' '08' '08' '08' '08' '08' '08' '08'
 '08' '08' '08' '08' '08' '08' '08' '08' '09' '09' '09' '09' '09' '09'
 '09' '09' '09' '09' '09' '09' '09' '09' '09' '10' '10' '10' '10' '10'
 '10' '10' '10' '10' '10' '10' '10' '10' '10' '10' '11' '11' '11' '11'
 '11' '11' '11' '11' '11' '11' '11' '11' '11' '11' '11' '12' '12' '12'
 '12' '12' '12' '12' '12' '12' '12' '12' '12' '12' '12' '12' '13' '13'
 '13' '1

In [205]:
P.shape

(570, 3072)

In [206]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
from glob import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Load Pretrained ResNet50 Model (without classification head)
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
# Data Augmentation
def extract_features(img_path, model):
    """Extract deep features from an image using ResNet50"""
    img = image.load_img(img_path, target_size=(224, 224))  # Resize to model input size
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)  # Normalize
    features = model.predict(img_array)  # Extract features
    return features.flatten()  # Convert to 1D array
# Load dataset (Change path to your dataset)
dataset_path = "C:\\Users\\prade\\OneDrive\\Desktop\\PROJECT\\Datasets\\V3-Y\\P"
classes = os.listdir(dataset_path)
X, y = [], []

for label, class_name in enumerate(classes):
    img_files = glob(os.path.join(dataset_path, class_name, "*.png"))  # Adjust for your format
    for img_path in img_files:
        features = extract_features(img_path, base_model)
        X.append(features)
        y.append(label)


# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)
P=X
print(P.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [207]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
from glob import glob

# Load Pretrained ResNet50 Model (without classification head)
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_features(img_path, model):
    """Extract deep features from an image using ResNet50"""
    img = image.load_img(img_path, target_size=(224, 224))  # Resize to model input size
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)  # Normalize
    features = model.predict(img_array)  # Extract features
    return features.flatten()  # Convert to 1D array

# Load dataset (Change path to your dataset)
dataset_path = "C:\\Users\\prade\\OneDrive\\Desktop\\PROJECT\\Datasets\\V3-Y\\t"
classes = os.listdir(dataset_path)
I, a = [], []

for label, class_name in enumerate(classes):
    img_files = glob(os.path.join(dataset_path, class_name, "*.png"))  # Adjust for your format
    for img_path in img_files:
        features = extract_features(img_path, base_model)
        I.append(features)
        a.append(label)

# Convert to NumPy arrays
I = np.array(I)
a = np.array(a)
print(I.shape)
print(a.shape)
t=I
print(t.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [208]:
P.shape

(570, 2048)

In [209]:
# Fisher Score function for feature selection
#import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from skfeature.function.similarity_based import fisher_score

# Assume X has shape (600, 3072) and y has corresponding labels
X = P  # Replace with actual dataset
y = train_labels  # Binary classification labels
# Split data into training and testing sets

# Ensure correct shapes
print("X shape:", X.shape)  # Should be (n_samples, n_features)
print("y shape:", y.shape)  # Should be (n_samples,)

# Apply Fisher Score
score = fisher_score.fisher_score(X, y)
print("Fisher Score Computed Successfully")
selected_features = np.argsort(score)[-1000:]  # Select top 50 features

P1 = X[:, selected_features]
print(P1)

X shape: (570, 2048)
y shape: (570,)
Fisher Score Computed Successfully
[[0.         0.228361   0.30172008 ... 0.9918864  0.00493643 1.3537453 ]
 [0.         0.08631588 0.15268344 ... 0.9167611  0.         1.4005588 ]
 [0.         0.10303758 0.88086    ... 0.70572454 0.0141919  0.6420712 ]
 ...
 [0.00805386 0.11457741 0.17418016 ... 0.2315948  0.01235673 0.47268388]
 [0.02370453 0.15200204 0.00358156 ... 0.06099871 0.         0.1852178 ]
 [0.01676682 0.14289486 0.03024724 ... 0.19347478 0.         0.5962264 ]]


In [210]:
t1=t[:,selected_features]
print(P.shape)

P1 = P1.T
print(P1.shape)
print(t1.shape)

(570, 2048)
(1000, 570)
(1710, 1000)


In [211]:
import numpy as np
from numpy.linalg import inv
from sklearn.preprocessing import OneHotEncoder

def initialize_variables(X, Y, c):
    d, n = X.shape  # d: features, n: samples
    encoder = OneHotEncoder(sparse_output=False, categories='auto')
    H = encoder.fit_transform(Y.reshape(-1, 1)).T  # One-hot encoding (c x n)
    #print(np.array2string(H, threshold=np.inf))
    ones_matrix = np.ones(H.shape)  # Shape: (c, n)
    B = 2 * H - ones_matrix  # Relaxation matrix (c x n)
    W = np.zeros((c, d))  # Projection matrix (c x d)

    # Change the shape of S to match the second dimension of B (n)
    S = np.zeros((c, n))  # Shape: (c, n)  # Updated to match B's shape
    print("H",H.shape)
    print("B",B.shape)
    print("S",S.shape)
    T = H + (B * S)  # Target matrix (c x n)
    E = np.zeros((c, n))  # Error matrix (c x n)
    R1 = np.zeros((c, n))  # Lagrange multiplier matrix (c x n)
    return W, S, T, E, R1, H, B

def fisher_criterion(T):
    c, n = T.shape  # Number of classes (rows) and samples per class (columns)
    M = np.mean(T, keepdims=True)  # Global mean as a scalar
    fisher_term = 0

    for i in range(c):
        Ti = T[i, :].reshape(1, -1)  # Ensure Ti is a 2D row vector
        Mi = np.mean(Ti, keepdims=True)  # Class mean as a scalar
        fisher_term += np.linalg.norm(Ti - Mi, 'fro')**2 - np.linalg.norm(Mi - M, 'fro')**2

    return fisher_term + np.linalg.norm(T, 'fro')**2

def update_T(W, X, H, B, S, E, R1, alpha, lambda_, mu):
    WX = W @ X
    M = np.mean(WX, axis=1, keepdims=True)
    M_hat = np.mean(WX, axis=1, keepdims=True)
    T = ((W @ X + alpha * (H + (B * S)) - lambda_ * (M - (2 * M_hat)) + (mu / 2) * (WX - E) + (R1 / 2))) / (1 + alpha +( 2 * lambda_ )+ (mu / 2))
    return T

def update_W(T, X, E, R1, beta, mu):
    XT = X.T
    W = (T + ((mu / 2) * (T + E)) + (R1 / 2)) @ XT @ inv(((mu / 2) + 1) * (X @ XT) + beta * np.eye(X.shape[0]))
    return W

def update_S(T, H, B):
    S = np.maximum(B * (T - H), 0)
    return S

def compute_E(gamma, Lambda_prime, mu, I, Q):
    term = inv(2 * gamma * Lambda_prime + (mu * I))
    E = term @ (mu * Q)
    return E

def compute_Lambda_prime(E, Delta):
    d, c = E.shape
    Lambda_prime = np.zeros((d, d))
    for i in range(d):
        sum_eij_squared = np.sum(E[i, :] ** 2)
        Lambda_prime[i, i] = 1 / (2 * np.sqrt(sum_eij_squared + Delta))
    return Lambda_prime

def update_E(W, X, T, R1, gamma, mu, Delta=1e-5):
    Q = W @ X - T + R1 / mu
    I = np.eye(Q.shape[0])
    Lambda_prime = compute_Lambda_prime(Q, Delta)
    E = compute_E(gamma, Lambda_prime, mu, I, Q)
    return E

def dcdlsr(X, Y, alpha=1, beta=0.01, lambda_=1, gamma=1, mu=1e-5, max_iter=10):
    c = np.unique(Y).size  # Number of classes
    W, S, T, E, R1, H, B = initialize_variables(X, Y, c)
    for _ in range(max_iter):
        T = update_T(W, X, H, B, S, E, R1, alpha, lambda_, mu)
        W = update_W(T, X, E, R1, beta, mu)
        S = update_S(T, H, B)
        E = update_E(W, X, T, R1, gamma, mu)
        R1 = R1 + (mu * (E - ((W @ X )- T)))
        mu = min(10, 1.1 * mu)
    return W, E

# Train DCDLSR model
W1, e = dcdlsr(P1,train_labels )
Q1=W1
print(W1)
print(e)
print(W1.shape)


H (38, 570)
B (38, 570)
S (38, 570)
[[ 0.01072275  0.01139596 -0.0037754  ... -0.00697466  0.01345859
   0.03344643]
 [ 0.02078439 -0.03026392 -0.02535553 ... -0.00904465 -0.00694884
  -0.01162297]
 [-0.01015451 -0.01354916 -0.01021361 ... -0.01089724 -0.00454089
  -0.02842581]
 ...
 [-0.02147781  0.01637027 -0.01046233 ... -0.01153414 -0.01120934
   0.00368588]
 [ 0.02608557 -0.02620445 -0.00945769 ...  0.0138463   0.02656652
  -0.00811726]
 [-0.02081024  0.03510791 -0.00629592 ... -0.02402361  0.01503952
  -0.01049563]]
[[ 5.91051223e-09  3.50279523e-09  6.16514641e-09 ... -1.63707955e-10
   1.25576568e-09 -3.30914604e-09]
 [-2.72989916e-09  7.27187874e-10  2.29935001e-09 ...  1.12797287e-09
  -2.07792807e-09  1.72841773e-09]
 [-1.75358913e-09 -1.69397344e-09  7.97536463e-10 ... -1.53125874e-09
  -1.29008701e-09  7.61436562e-10]
 ...
 [ 6.61067918e-11 -7.27789963e-11 -5.30351740e-10 ... -5.98472342e-10
   1.27150618e-10 -5.66213792e-10]
 [-1.94471255e-10 -3.70126348e-10 -7.68492808e-

In [212]:
# Define image size (resize all images to this size)
IMAGE_SIZE = (32,32)  # You can change this

# Get list of subfolders (each subfolder is a dog breed)
face = sorted([folder.name for folder in d_p.iterdir() if folder.is_dir()])
print("Face Found:", face)

# Initialize empty lists to store data

Z = []
# Loop through each breed (subfolder)
for f in face:
    f_path = d_p / f

    for img_file in f_path.glob("*.png"):  # Adjust file extensions if needed
        # Read and resize image
        img = cv2.imread(str(img_file))  # Read image
        img = cv2.resize(img, IMAGE_SIZE)  # Resize to fixed size
        img = img / 255.0  # Normalize pixel values (0 to 1)
        # Append corresponding label
        Z.append(f)
# Convert lists to NumPy arrays

print(P.shape)
Z = np.array(Z)
Z.shape
# Encode labels as numbers
label_encoder = LabelEncoder()
Z_encoded = label_encoder.fit_transform(Z)  # Converts breed names to integer labels
print(f"Target Matrix Z Shape: {Z_encoded.shape}")  # (n,)
print(f"Unique Labels: {len(np.unique(Z_encoded))}")

Face Found: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39']
(570, 2048)
Target Matrix Z Shape: (2280,)
Unique Labels: 38


In [213]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
# P: ResNet features from training images
# t: ResNet features from test images
# train_labels: labels for training set
# test_labels: labels for test set

selector = SelectKBest(score_func=f_classif, k=1000)
selector.fit(P, train_labels)

# Step 2: Select top features from both train and test sets
P2 = selector.transform(P)
t2 = selector.transform(t)

print("Selected Feature Mask:", selector.get_support())   # Boolean mask of selected features
print("P2 shape (train with selected features):", P2.shape)
print("t2 shape (test with selected features):", t2.shape)
# Optional: Transpose if your model expects features along rows
P2 = P2.T
t2 = t2
print("P2 shape (train with selected features):", P2.shape)
print("t2 shape (test with selected features):", t2.shape)

Selected Feature Mask: [False  True False ...  True False False]
P2 shape (train with selected features): (570, 1000)
t2 shape (test with selected features): (1710, 1000)
P2 shape (train with selected features): (1000, 570)
t2 shape (test with selected features): (1710, 1000)


In [214]:
import numpy as np
from numpy.linalg import inv
from sklearn.preprocessing import OneHotEncoder

def initialize_variables(X, Y, c):
    d, n = X.shape  # d: features, n: samples
    encoder = OneHotEncoder(sparse_output=False, categories='auto')
    H = encoder.fit_transform(Y.reshape(-1, 1)).T  # One-hot encoding (c x n)
    #print(np.array2string(H, threshold=np.inf))
    ones_matrix = np.ones(H.shape)  # Shape: (c, n)
    B = 2 * H - ones_matrix  # Relaxation matrix (c x n)
    W = np.zeros((c, d))  # Projection matrix (c x d)

    # Change the shape of S to match the second dimension of B (n)
    S = np.zeros((c, n))  # Shape: (c, n)  # Updated to match B's shape
    print("H",H.shape)
    print("B",B.shape)
    print("S",S.shape)
    T = H + (B * S)  # Target matrix (c x n)
    E = np.zeros((c, n))  # Error matrix (c x n)
    R1 = np.zeros((c, n))  # Lagrange multiplier matrix (c x n)
    return W, S, T, E, R1, H, B

def fisher_criterion(T):
    c, n = T.shape  # Number of classes (rows) and samples per class (columns)
    M = np.mean(T, keepdims=True)  # Global mean as a scalar
    fisher_term = 0

    for i in range(c):
        Ti = T[i, :].reshape(1, -1)  # Ensure Ti is a 2D row vector
        Mi = np.mean(Ti, keepdims=True)  # Class mean as a scalar
        fisher_term += np.linalg.norm(Ti - Mi, 'fro')**2 - np.linalg.norm(Mi - M, 'fro')**2

    return fisher_term + np.linalg.norm(T, 'fro')**2

def update_T(W, X, H, B, S, E, R1, alpha, lambda_, mu):
    WX = W @ X
    M = np.mean(WX, axis=1, keepdims=True)
    M_hat = np.mean(WX, axis=1, keepdims=True)
    T = ((W @ X + alpha * (H + (B * S)) - lambda_ * (M - (2 * M_hat)) + (mu / 2) * (WX - E) + (R1 / 2))) / (1 + alpha +( 2 * lambda_ )+ (mu / 2))
    return T

def update_W(T, X, E, R1, beta, mu):
    XT = X.T
    W = (T + ((mu / 2) * (T + E)) + (R1 / 2)) @ XT @ inv(((mu / 2) + 1) * (X @ XT) + beta * np.eye(X.shape[0]))
    return W

def update_S(T, H, B):
    S = np.maximum(B * (T - H), 0)
    return S

def compute_E(gamma, Lambda_prime, mu, I, Q):
    term = inv(2 * gamma * Lambda_prime + (mu * I))
    E = term @ (mu * Q)
    return E

def compute_Lambda_prime(E, Delta):
    d, c = E.shape
    Lambda_prime = np.zeros((d, d))
    for i in range(d):
        sum_eij_squared = np.sum(E[i, :] ** 2)
        Lambda_prime[i, i] = 1 / (2 * np.sqrt(sum_eij_squared + Delta))
    return Lambda_prime

def update_E(W, X, T, R1, gamma, mu, Delta=1e-5):
    Q = W @ X - T + R1 / mu
    I = np.eye(Q.shape[0])
    Lambda_prime = compute_Lambda_prime(Q, Delta)
    E = compute_E(gamma, Lambda_prime, mu, I, Q)
    return E

def dcdlsr(X, Y, alpha=1, beta=0.01, lambda_=1, gamma=1, mu=1e-5, max_iter=10):
    c = np.unique(Y).size  # Number of classes
    W, S, T, E, R1, H, B = initialize_variables(X, Y, c)
    for _ in range(max_iter):
        T = update_T(W, X, H, B, S, E, R1, alpha, lambda_, mu)
        W = update_W(T, X, E, R1, beta, mu)
        S = update_S(T, H, B)
        E = update_E(W, X, T, R1, gamma, mu)
        R1 = R1 + (mu * (E - ((W @ X )- T)))
        mu = min(10, 1.1 * mu)
    return W, E

# Train DCDLSR model
W2, e = dcdlsr(P2,train_labels )
Q2=W2
print(W2)
print(e)
print(W2.shape)


H (38, 570)
B (38, 570)
S (38, 570)
[[ 0.01102314  0.00076884  0.00795627 ... -0.00277961 -0.00917139
  -0.00902359]
 [ 0.00237351 -0.00171726  0.00482408 ... -0.00840068 -0.00658694
   0.00745813]
 [-0.0193403  -0.00383647 -0.00194223 ... -0.00565994  0.00881263
   0.00889612]
 ...
 [ 0.00349009  0.00524869  0.00342642 ...  0.01587491 -0.00713889
   0.01633402]
 [ 0.01338552 -0.0004131   0.0022112  ...  0.00182103 -0.00047592
  -0.00353593]
 [ 0.00070848 -0.01020651 -0.0014076  ... -0.00827442 -0.0060812
  -0.0092489 ]]
[[ 3.93874239e-10  3.93435541e-10  6.57951484e-10 ... -4.22749295e-12
  -5.42734105e-11 -2.78485340e-10]
 [-2.85743390e-10  1.11988984e-10  3.96775314e-11 ... -1.89192802e-10
  -2.66704185e-10  5.96628342e-10]
 [ 1.91896874e-10 -2.69444576e-10 -3.41941377e-10 ...  5.31200813e-11
  -2.72076744e-10  3.84051216e-10]
 ...
 [-1.58614615e-10 -1.16842948e-11  2.70513101e-10 ... -1.19518911e-10
  -1.43909610e-10  9.21482489e-11]
 [-4.55852531e-10  1.05028668e-10 -1.37960841e-1

In [215]:
Qsum = np.vstack((Q1, Q2)) # Concatenate along columns
print("Concatenated Optimum Matrix Shape:", Qsum.shape)
print(Qsum)

Psum = sum((P1, P2)) # Concatenate along columns
print("Concatenated Optimum Matrix Shape:", Psum.shape)
print(Psum)
tsum = sum((t1, t2)) # Concatenate along columns
print("Concatenated Optimum Matrix Shape:", Psum.shape)
print(tsum)

Concatenated Optimum Matrix Shape: (76, 1000)
[[ 0.01072275  0.01139596 -0.0037754  ... -0.00697466  0.01345859
   0.03344643]
 [ 0.02078439 -0.03026392 -0.02535553 ... -0.00904465 -0.00694884
  -0.01162297]
 [-0.01015451 -0.01354916 -0.01021361 ... -0.01089724 -0.00454089
  -0.02842581]
 ...
 [ 0.00349009  0.00524869  0.00342642 ...  0.01587491 -0.00713889
   0.01633402]
 [ 0.01338552 -0.0004131   0.0022112  ...  0.00182103 -0.00047592
  -0.00353593]
 [ 0.00070848 -0.01020651 -0.0014076  ... -0.00827442 -0.0060812
  -0.0092489 ]]
Concatenated Optimum Matrix Shape: (1000, 570)
[[0.07217308 0.05398307 0.2953569  ... 0.27928802 0.16750425 0.16388646]
 [1.0988998  1.3518089  0.6406187  ... 2.1079192  1.6640229  1.8942083 ]
 [2.8362699  2.8243308  3.1925673  ... 4.9084945  3.027199   4.088154  ]
 ...
 [1.2015796  1.8586967  1.4808342  ... 3.237908   3.0334802  3.3958433 ]
 [0.21695037 0.56574726 0.34347352 ... 0.27509156 0.5713114  0.554686  ]
 [1.474577   1.4005588  0.7303605  ... 0.52684

In [216]:
P_cap = Qsum @ Psum
t_cap = Qsum @ tsum.T
print(P_cap.shape)
print(t_cap.shape)

(76, 570)
(76, 1710)


In [217]:
from collections import Counter

k =  3
y_true = []
y_pred = []
# Initialize accuracy tracking
correct_predictions = 0
total_predictions = 0

# Loop through each test sample
for i in range(t_cap.shape[1]):  # 150 test samples
    test_image = t_cap[:, i]  # Get the ith test image

    # === Step 1: Compute Euclidean Distance ===
    distances = np.linalg.norm(P_cap - test_image[:, np.newaxis], axis=0)  # (600,)
    #print(distances)
    # === Step 2: Find k Nearest Neighbors ===
    nearest_indices = np.argsort(distances)[:k]  # Get indices of k nearest neighbors
    nearest_classes = train_labels[nearest_indices]  # Get the corresponding classes

    # === Step 3: Majority Voting ===
    predicted_class = Counter(nearest_classes).most_common(1)[0][0]
    expected_class = test_labels[i]
    y_true.append(expected_class)
    y_pred.append(predicted_class)
    # === Step 5: Accuracy Calculation ===
    if predicted_class == expected_class:
        correct_predictions += 1
    total_predictions += 1

# === Step 6: Overall Accuracy ===
accuracy = (correct_predictions / total_predictions) * 100
print(f"Overall Accuracy: {accuracy:.2f}%")
print(Num_of_train_image)

Overall Accuracy: 73.04%
15


In [218]:
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Example true labels and predicted labels


# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)
labels = ["Class 0", "Class 1"]  # Modify for multi-class problems
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)


# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Compute precision
precision = precision_score(y_true, y_pred, average='weighted')
print("Precision:", precision)

# Compute recall
recall = recall_score(y_true, y_pred, average='weighted')
print("Recall:", recall)

# Compute F1-score
f1 = f1_score(y_true, y_pred, average='weighted')
print("F1 Score:", f1)
# Plot confusion matrix

Confusion Matrix:
 [[34  0  2 ...  0  0  0]
 [ 0 35  1 ...  0  0  0]
 [ 2  0 28 ...  0  3  0]
 ...
 [ 0  0  0 ... 40  0  0]
 [ 0  1  0 ...  0 30  0]
 [ 0  0  0 ...  0  0 39]]
Accuracy: 0.7304093567251462
Precision: 0.7495753795887603
Recall: 0.7304093567251462
F1 Score: 0.7336507209748708
